In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [28]:
path = '/content/drive/MyDrive/2조self/Cream_dataset1_5sample.csv'
raw = pd.read_csv(path, index_col=0)
cream = raw.copy()
cream[:1]

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,[굿즈증정/1+1]닥터지 레드 블레미쉬 클리어 수딩 크림 70ml 1+1 기획,닥터지,"38,000","28,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,70ml+70ml+인사이드아웃 콜라보 시온컵,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...","[['여름용 크림으로 라이트한 걸 찾던 중 샀어요. 아침용으로 쓰기 좋아요'], [..."


In [29]:
cream.review

0      [['여름용 크림으로 라이트한 걸 찾던 중 샀어요. 아침용으로 쓰기 좋아요'], [...
1      [['[옵션] 시카플라스트밤 100ml 기획'], ['[옵션] [기획] 시카밤 10...
2      [['무겁지 않고 수분감을 가득 채워줘서 여름에도 편하게 사용하기 좋아요'], ['...
3      [['리뷰가 좋길래 한번 사봤어요!\n제발 잘 맞았으면 좋겠어요ㅠ'], ['다른 제...
4      [['쿨링감도 있고 피부에 흡수도 잘되고 자극없이\n사용할수 있어서 좋아요.\n생각...
                             ...                        
170    [['운동하고나서 꼭 바르는 수분크림! 벌써 몇통이나 비웟는지 몰라요 바르고 나면 ...
171    [['이거로 제 료션은 정착합니다 ㅎㅎ 뭐하나 빠지는게 있다면 유분기에요 ㅠㅋㅋ']...
172    [['건성보다는 복합성이 좋은거같아요 기능성이다보니 매일은 사용하지않고 보습이 짱짱...
173    [['화장도 안 밀리고 여드름성 피부들이 바르기에 딱 좋은 것 같아요!'], ['남...
174    [['아이크림 유명한거 대부분 다 써봣는데 얘는 진짜 주름이 없어져요,,,, 진짜 ...
Name: review, Length: 175, dtype: object

In [30]:
cream.shape

(175, 10)

In [31]:
cream = cream.drop_duplicates(subset=('name','review'), keep='first')
cream.shape

(175, 10)

In [32]:
duplicate_data = cream[cream.duplicated(subset='name', keep=False)]
# keep=False 옵션을 사용하여 모든 중복된 행을 선택
duplicate_data

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review


In [33]:
# 동일 제품의 리뷰 확인할 결과 대부분의 내용 동일
cream = cream.drop_duplicates(subset=('name'), keep='first')
cream.shape

(175, 10)

In [34]:
# 'review' 열에서 '], ['를 기준으로 문자열 분리
def split_reviews(review_str):
    # 문자열에서 '], ['를 기준으로 분리한 후, 앞뒤의 대괄호 제거
    if isinstance(review_str, str):
        # '], ['를 기준으로 분리
        parts = review_str.split("'], ['")
        # 대괄호 제거
        parts = [part.strip("[]'") for part in parts]
        return parts
    else:
        return []
# 'review' 열에 함수 적용
cream['review'] = cream['review'].apply(split_reviews)

In [35]:
cream[['review']]

,review
0,"[여름용 크림으로 라이트한 걸 찾던 중 샀어요. 아침용으로 쓰기 좋아요, 좋아요. ..."
1,"[옵션] 시카플라스트밤 100ml 기획, 옵션] [기획] 시카밤 100ml SET ..."
2,"[무겁지 않고 수분감을 가득 채워줘서 여름에도 편하게 사용하기 좋아요, 유분이 좀 ..."
3,"[리뷰가 좋길래 한번 사봤어요!\n제발 잘 맞았으면 좋겠어요ㅠ, 다른 제품 쓰다가 ..."
4,[쿨링감도 있고 피부에 흡수도 잘되고 자극없이\n사용할수 있어서 좋아요.\n생각보다...
...,...
170,[운동하고나서 꼭 바르는 수분크림! 벌써 몇통이나 비웟는지 몰라요 바르고 나면 짱 ...
171,"[이거로 제 료션은 정착합니다 ㅎㅎ 뭐하나 빠지는게 있다면 유분기에요 ㅠㅋㅋ, 피부..."
172,[건성보다는 복합성이 좋은거같아요 기능성이다보니 매일은 사용하지않고 보습이 짱짱한것...
173,"[화장도 안 밀리고 여드름성 피부들이 바르기에 딱 좋은 것 같아요!, 남자친구가 사..."


In [36]:
cream.review[1][0]

'옵션] 시카플라스트밤 100ml 기획'

In [37]:
# 각 리뷰를 별도의 행으로 변환하는 함수 정의
def explode_reviews(df):
    # DataFrame의 각 행에 대해 'review' 열을 열의 요소로 분리하여 새로운 행 생성
    return df.explode('review')

# 변환 적용
expanded_cream_data = explode_reviews(cream)
expanded_cream_data

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,[굿즈증정/1+1]닥터지 레드 블레미쉬 클리어 수딩 크림 70ml 1+1 기획,닥터지,"38,000","28,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,70ml+70ml+인사이드아웃 콜라보 시온컵,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...",여름용 크림으로 라이트한 걸 찾던 중 샀어요. 아침용으로 쓰기 좋아요
0,[굿즈증정/1+1]닥터지 레드 블레미쉬 클리어 수딩 크림 70ml 1+1 기획,닥터지,"38,000","28,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,70ml+70ml+인사이드아웃 콜라보 시온컵,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...",좋아요. 옛날에 써보고 괜찮아서 재구매했는데 나쁘지 않아요. 튜브형으로도 나오면 좋...
0,[굿즈증정/1+1]닥터지 레드 블레미쉬 클리어 수딩 크림 70ml 1+1 기획,닥터지,"38,000","28,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,70ml+70ml+인사이드아웃 콜라보 시온컵,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...",이거 지금 3통째 쓰는 중인데. 순하고 너무 좋아요 크케 진정 효과는 없지만 피부가...
0,[굿즈증정/1+1]닥터지 레드 블레미쉬 클리어 수딩 크림 70ml 1+1 기획,닥터지,"38,000","28,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,70ml+70ml+인사이드아웃 콜라보 시온컵,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...",보통 로션 생각하면 지성들은 유분기 있어서 잘 못쓰는 경우 있는데 ( 제 경우엔) ...
0,[굿즈증정/1+1]닥터지 레드 블레미쉬 클리어 수딩 크림 70ml 1+1 기획,닥터지,"38,000","28,800",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,70ml+70ml+인사이드아웃 콜라보 시온컵,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜, 다이카프릴릴에터, 나이아신아마이드, 1,2-헥산...",순하고 보습에도 괜찮아요! 2개 저렴하게\n구매할 수 있어서 만족스러워요!
...,...,...,...,...,...,...,...,...,...,...
174,OOTD AM 인텐스 아이세럼 30ml,OOTD,"33,000","17,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30g,모든 피부,"정제수, 글리세린, 프로필렌글라이콜, 세틸에틸헥사노에이트, 하이드로제네이티드폴리데센...",눈 부분이 건조해져서 구입했어요~\n생각보다 발림성 좋고 촉촉해져서 좋네요\n건조한...
174,OOTD AM 인텐스 아이세럼 30ml,OOTD,"33,000","17,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30g,모든 피부,"정제수, 글리세린, 프로필렌글라이콜, 세틸에틸헥사노에이트, 하이드로제네이티드폴리데센...",아이세럼 이것저것 써보고 드디어 정착 했어요 피부에 자극적이지 않고 끈적임 없이 발...
174,OOTD AM 인텐스 아이세럼 30ml,OOTD,"33,000","17,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30g,모든 피부,"정제수, 글리세린, 프로필렌글라이콜, 세틸에틸헥사노에이트, 하이드로제네이티드폴리데센...",세안 후 아이크림 아이세럼등등\n보습과 주름에 관련된 제품들을 꾸준히 써왔습니다\n...
174,OOTD AM 인텐스 아이세럼 30ml,OOTD,"33,000","17,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30g,모든 피부,"정제수, 글리세린, 프로필렌글라이콜, 세틸에틸헥사노에이트, 하이드로제네이티드폴리데센...",여름에 가볍게 바를 수 있는 산뜻한 세럼이네요 아이크림 바르고 화장하면 유분기때문에...


In [38]:
# 제품별 리뷰 개수
cream_count = expanded_cream_data[['name','review']].groupby('name').size().reset_index(name='count')
# cream_count[cream_count['count'] < 80]
cream_count

,name,count
0,OOTD AM 인텐스 아이세럼 30ml,100
1,[1&1/10년연속1위] 유세린 하이알루론 3X 아이크림 더블기획 (15ml+15ml),99
2,[1&1] 유세린 울트라센시티브 리페어 크림 50ml 더블기획 (50ml+50ml),100
3,[1+1 기획] 차앤박(CNP) 아쿠아 수딩 크림 1+1 기획,100
4,[1+1/건성피부] 유세린 아쿠아포린 액티브 리치크림 (50ml+50ml),100
...,...,...
170,피지오겔 DMT 인텐시브크림 100ml,98
171,피지오겔 DMT 페이셜 수분크림 150ml,100
172,피지오겔 레드수딩 시카밸런스 크림 80ml 기획,100
173,한율 빨간쌀 보습탄력 크림 55ml,100


In [39]:
# 80개 미만 댓글을 가진 제품 없음
cream_count[cream_count['count'] < 80]

,name,count


In [40]:
expanded_cream_data.shape

(17338, 10)

In [46]:
# 동일 제품 내에 동일한 리뷰 확인
expanded_cream_data = expanded_cream_data.drop_duplicates(subset=('name', 'review'))
expanded_cream_data = expanded_cream_data[['name', 'review']]
expanded_cream_data.shape

(16806, 2)

In [47]:
import pandas as pd
# # 모든 열 너비를 표시
#pd.set_option('display.max_colwidth', None)
# # 옵션을 기본값으로 재설정
#pd.reset_option('display.max_colwidth')

In [48]:
# 제거한 후, 동일 제품 내 리뷰 개수
expanded_cream_data_count = expanded_cream_data[['name','review']].groupby('name').size().reset_index(name='count')
expanded_cream_data_count[expanded_cream_data_count['count']>80].name

0                                 OOTD AM 인텐스 아이세럼 30ml
1      [1&1/10년연속1위] 유세린 하이알루론 3X 아이크림 더블기획 (15ml+15ml)
2        [1&1] 유세린 울트라센시티브 리페어 크림 50ml 더블기획 (50ml+50ml)
3                    [1+1 기획] 차앤박(CNP) 아쿠아 수딩 크림 1+1 기획
4             [1+1/건성피부] 유세린 아쿠아포린 액티브 리치크림 (50ml+50ml)
                             ...                       
170                               피지오겔 DMT 인텐시브크림 100ml
171                             피지오겔 DMT 페이셜 수분크림 150ml
172                          피지오겔 레드수딩 시카밸런스 크림 80ml 기획
173                                 한율 빨간쌀 보습탄력 크림 55ml
174                            헤브블루 살몬 케어링 센텔라 크림 100ml
Name: name, Length: 175, dtype: object

In [49]:
expanded_cream_data[['name','review']].groupby('name').size().reset_index(name='count')

,name,count
0,OOTD AM 인텐스 아이세럼 30ml,100
1,[1&1/10년연속1위] 유세린 하이알루론 3X 아이크림 더블기획 (15ml+15ml),98
2,[1&1] 유세린 울트라센시티브 리페어 크림 50ml 더블기획 (50ml+50ml),100
3,[1+1 기획] 차앤박(CNP) 아쿠아 수딩 크림 1+1 기획,99
4,[1+1/건성피부] 유세린 아쿠아포린 액티브 리치크림 (50ml+50ml),100
...,...,...
170,피지오겔 DMT 인텐시브크림 100ml,92
171,피지오겔 DMT 페이셜 수분크림 150ml,90
172,피지오겔 레드수딩 시카밸런스 크림 80ml 기획,98
173,한율 빨간쌀 보습탄력 크림 55ml,97


In [50]:
# # 제거할 'name' 목록 정의
# names_to_remove = [
#     '[1ST 보습케어] 달바 화이트 트러플 퍼스트 아로마틱 토너 155ml',
#     '[단독기획] 아누아 어성초 77 수딩 토너 350ml 리필 기획세트(+350ml 리필팩)',
#     '[단독기획] 토니모리 원더 세라마이드 모찌 토너 (본품500ml+100ml)',
#     '코스알엑스 하이드리움 워터리 토너 280ml'
# ]

# # 'name' 열의 값이 names_to_remove에 포함되지 않은 행들로 새로운 DataFrame 생성
# expanded_cream_data = expanded_cream_data[~expanded_cream_data['name'].isin(names_to_remove)]

# # 결과 확인
# expanded_cream_data.shape


In [51]:
def collapse_review(df): # 행으로 변환한 리뷰를 다시 하나의 리스트 형식으로
  return df.groupby('name', as_index = False).agg({'review': lambda x : list(x)})

cream_data = collapse_review(expanded_cream_data)
cream_data.shape

(175, 2)

In [52]:
cream_data[:1]

,name,review
0,OOTD AM 인텐스 아이세럼 30ml,"[아이크림 유명한거 대부분 다 써봣는데 얘는 진짜 주름이 없어져요,,,, 진짜 대박..."


In [53]:
# 각 제품의 리뷰 수를 80개로 맞추는 함수 정의
def sample_reviews(reviews, n=80):
    if len(reviews) > n:
        return pd.Series(reviews).sample(n=n, random_state=1).tolist()
    else:
        return reviews


In [54]:
cream_data['review'] = cream_data['review'].apply(sample_reviews)

In [56]:
len(cream_data['review'][4])

80

In [57]:
cream_data

,name,review
0,OOTD AM 인텐스 아이세럼 30ml,[펌핑하게 되어있고 뚜껑으로 한번 더 막을 수 있어서 위생적으로 쓸 수 있겠더라구요...
1,[1&1/10년연속1위] 유세린 하이알루론 3X 아이크림 더블기획 (15ml+15ml),"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
2,[1&1] 유세린 울트라센시티브 리페어 크림 50ml 더블기획 (50ml+50ml),"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
3,[1+1 기획] 차앤박(CNP) 아쿠아 수딩 크림 1+1 기획,[옵션] 80ml+80ml 남자가 사용하기에도 끈적이지 않고 좋은것같아요.\n흡수도...
4,[1+1/건성피부] 유세린 아쿠아포린 액티브 리치크림 (50ml+50ml),"[우리엄마 인생탬 다른거 사줄까? 해도 무조건 이 크림이랍니다ㅋ, 수분감과 영양감 ..."
...,...,...
170,피지오겔 DMT 인텐시브크림 100ml,"[피지오겔 .. 가격만 적절하면 왕창왕창 쓰고 싶어요 항상 쓰는 제품입니다, 항상 ..."
171,피지오겔 DMT 페이셜 수분크림 150ml,[옵션] 피지오겔 DMT 페이셜크림 친구 추천으로 샀어요 너무 좋더라구용 재구매 의...
172,피지오겔 레드수딩 시카밸런스 크림 80ml 기획,[발림성이 좋은 것 같아요. 흡수도 빨랐구요. 위에 적혀있는 것처럼 붉은기에 개선이...
173,한율 빨간쌀 보습탄력 크림 55ml,"[엄마가 쓸 크림 찾다가 구매항건데 보습력 괜찮네요! 만족합니다, 피부가 너무 건조..."


In [58]:
cream_data.columns = ['name', 'sample_review']

In [59]:
cream_data

,name,sample_review
0,OOTD AM 인텐스 아이세럼 30ml,[펌핑하게 되어있고 뚜껑으로 한번 더 막을 수 있어서 위생적으로 쓸 수 있겠더라구요...
1,[1&1/10년연속1위] 유세린 하이알루론 3X 아이크림 더블기획 (15ml+15ml),"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
2,[1&1] 유세린 울트라센시티브 리페어 크림 50ml 더블기획 (50ml+50ml),"[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
3,[1+1 기획] 차앤박(CNP) 아쿠아 수딩 크림 1+1 기획,[옵션] 80ml+80ml 남자가 사용하기에도 끈적이지 않고 좋은것같아요.\n흡수도...
4,[1+1/건성피부] 유세린 아쿠아포린 액티브 리치크림 (50ml+50ml),"[우리엄마 인생탬 다른거 사줄까? 해도 무조건 이 크림이랍니다ㅋ, 수분감과 영양감 ..."
...,...,...
170,피지오겔 DMT 인텐시브크림 100ml,"[피지오겔 .. 가격만 적절하면 왕창왕창 쓰고 싶어요 항상 쓰는 제품입니다, 항상 ..."
171,피지오겔 DMT 페이셜 수분크림 150ml,[옵션] 피지오겔 DMT 페이셜크림 친구 추천으로 샀어요 너무 좋더라구용 재구매 의...
172,피지오겔 레드수딩 시카밸런스 크림 80ml 기획,[발림성이 좋은 것 같아요. 흡수도 빨랐구요. 위에 적혀있는 것처럼 붉은기에 개선이...
173,한율 빨간쌀 보습탄력 크림 55ml,"[엄마가 쓸 크림 찾다가 구매항건데 보습력 괜찮네요! 만족합니다, 피부가 너무 건조..."


In [60]:
merge_cream = pd.merge(cream, cream_data, on='name', how='right')
merge_cream.columns

Index(['name', 'brand', 'price', 'sale_price', 'picture', 'url', 'volume',
       'skin_type', 'ingredient', 'review', 'sample_review'],
      dtype='object')

In [61]:
merge_cream.shape

(175, 11)

In [62]:
merge_cream = merge_cream[['name', 'brand', 'price', 'sale_price', 'picture', 'url', 'volume',
       'cream_type', 'ingredient','sample_review']]

In [63]:
merge_cream.columns = ['name', 'brand', 'price', 'sale_price', 'picture', 'url', 'volume',
       'cream_type', 'ingredient','review']

In [64]:
merge_cream

,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,OOTD AM 인텐스 아이세럼 30ml,OOTD,"33,000","17,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30g,모든 피부,"정제수, 글리세린, 프로필렌글라이콜, 세틸에틸헥사노에이트, 하이드로제네이티드폴리데센...",[펌핑하게 되어있고 뚜껑으로 한번 더 막을 수 있어서 위생적으로 쓸 수 있겠더라구요...
1,[1&1/10년연속1위] 유세린 하이알루론 3X 아이크림 더블기획 (15ml+15ml),유세린,"75,900",NaN,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,15ml/2입,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜다이카프릴레이트/다이카프레이트, 디에칠아미노하이드록...","[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
2,[1&1] 유세린 울트라센시티브 리페어 크림 50ml 더블기획 (50ml+50ml),유세린,"49,000",NaN,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,50ml*2,모든 피부용,"정제수,글리세린,판테놀,시어버터,세틸팔미테이트,메틸팔미테이트,식물성오일,펜틸렌글라이...","[※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별 ..."
3,[1+1 기획] 차앤박(CNP) 아쿠아 수딩 크림 1+1 기획,차앤박,"33,000",NaN,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,80ml,모든 피부용,"정제수, 부틸렌글라이콜, 다이메티콘, 글리세린, 1,2-헥산다이올, 사이클로펜타실록...",[옵션] 80ml+80ml 남자가 사용하기에도 끈적이지 않고 좋은것같아요.\n흡수도...
4,[1+1/건성피부] 유세린 아쿠아포린 액티브 리치크림 (50ml+50ml),유세린,"42,000","39,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,50ml/2입,건성 피부용,"정제수,글리세린,글리세릴글루코사이드,호호바씨오일,하이드로제네이티드코코-글리세라이즈,...","[우리엄마 인생탬 다른거 사줄까? 해도 무조건 이 크림이랍니다ㅋ, 수분감과 영양감 ..."
...,...,...,...,...,...,...,...,...,...,...
170,피지오겔 DMT 인텐시브크림 100ml,피지오겔,"46,500","26,910",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,■100ml,■모든 피부용,"■정제수, 카프릴릭/카프릭트리글리세라이드, 쉐어버터,\n글리세린, 펜틸렌글라이콜, ...","[피지오겔 .. 가격만 적절하면 왕창왕창 쓰고 싶어요 항상 쓰는 제품입니다, 항상 ..."
171,피지오겔 DMT 페이셜 수분크림 150ml,피지오겔,"47,500","26,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,■본품 150ml,■모든 피부용,"■정제수, 카프릴릭/카프릭트리글리세라이드,, 글리세린, 펜틸렌글라이콜, 코코넛오일,...",[옵션] 피지오겔 DMT 페이셜크림 친구 추천으로 샀어요 너무 좋더라구용 재구매 의...
172,피지오겔 레드수딩 시카밸런스 크림 80ml 기획,피지오겔,"30,000","21,250",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,80ml,모든피부용.,"정제수, 하이드로제네이티드폴리아이소부텐, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,...",[발림성이 좋은 것 같아요. 흡수도 빨랐구요. 위에 적혀있는 것처럼 붉은기에 개선이...
173,한율 빨간쌀 보습탄력 크림 55ml,한율,"49,000","41,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,55ml,모든 피부,"정제수, 부틸렌글라이콜, 세틸에틸헥사노에이트, 글리세린, 펜타에리스리틸테트라에틸헥사...","[엄마가 쓸 크림 찾다가 구매항건데 보습력 괜찮네요! 만족합니다, 피부가 너무 건조..."


In [65]:
path = '/content/drive/MyDrive/2조self/Cream_datase1_5t.csv'
merge_cream.to_csv(path)

In [66]:
pd.read_csv(path)

,Unnamed: 0,name,brand,price,sale_price,picture,url,volume,skin_type,ingredient,review
0,0,OOTD AM 인텐스 아이세럼 30ml,OOTD,"33,000","17,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,30g,모든 피부,"정제수, 글리세린, 프로필렌글라이콜, 세틸에틸헥사노에이트, 하이드로제네이티드폴리데센...",['펌핑하게 되어있고 뚜껑으로 한번 더 막을 수 있어서 위생적으로 쓸 수 있겠더라구...
1,1,[1&1/10년연속1위] 유세린 하이알루론 3X 아이크림 더블기획 (15ml+15ml),유세린,"75,900",NaN,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,15ml/2입,모든 피부용,"정제수, 글리세린, 부틸렌글라이콜다이카프릴레이트/다이카프레이트, 디에칠아미노하이드록...","['※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
2,2,[1&1] 유세린 울트라센시티브 리페어 크림 50ml 더블기획 (50ml+50ml),유세린,"49,000",NaN,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,50ml*2,모든 피부용,"정제수,글리세린,판테놀,시어버터,세틸팔미테이트,메틸팔미테이트,식물성오일,펜틸렌글라이...","['※ 해당 리뷰는 원칙적으로 기본 상품이 동일한 단품 사용 후 작성된 것이며,개별..."
3,3,[1+1 기획] 차앤박(CNP) 아쿠아 수딩 크림 1+1 기획,차앤박,"33,000",NaN,https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,80ml,모든 피부용,"정제수, 부틸렌글라이콜, 다이메티콘, 글리세린, 1,2-헥산다이올, 사이클로펜타실록...",['옵션] 80ml+80ml 남자가 사용하기에도 끈적이지 않고 좋은것같아요.\\n흡...
4,4,[1+1/건성피부] 유세린 아쿠아포린 액티브 리치크림 (50ml+50ml),유세린,"42,000","39,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,50ml/2입,건성 피부용,"정제수,글리세린,글리세릴글루코사이드,호호바씨오일,하이드로제네이티드코코-글리세라이즈,...","['우리엄마 인생탬 다른거 사줄까? 해도 무조건 이 크림이랍니다ㅋ', '수분감과 영..."
...,...,...,...,...,...,...,...,...,...,...,...
170,170,피지오겔 DMT 인텐시브크림 100ml,피지오겔,"46,500","26,910",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,■100ml,■모든 피부용,"■정제수, 카프릴릭/카프릭트리글리세라이드, 쉐어버터,\n글리세린, 펜틸렌글라이콜, ...","['피지오겔 .. 가격만 적절하면 왕창왕창 쓰고 싶어요 항상 쓰는 제품입니다', '..."
171,171,피지오겔 DMT 페이셜 수분크림 150ml,피지오겔,"47,500","26,900",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,■본품 150ml,■모든 피부용,"■정제수, 카프릴릭/카프릭트리글리세라이드,, 글리세린, 펜틸렌글라이콜, 코코넛오일,...",['옵션] 피지오겔 DMT 페이셜크림 친구 추천으로 샀어요 너무 좋더라구용 재구매 ...
172,172,피지오겔 레드수딩 시카밸런스 크림 80ml 기획,피지오겔,"30,000","21,250",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,80ml,모든피부용.,"정제수, 하이드로제네이티드폴리아이소부텐, 부틸렌글라이콜, 다이프로필렌글라이콜, 1,...",['발림성이 좋은 것 같아요. 흡수도 빨랐구요. 위에 적혀있는 것처럼 붉은기에 개선...
173,173,한율 빨간쌀 보습탄력 크림 55ml,한율,"49,000","41,600",https://image.oliveyoung.co.kr/uploads/images/...,https://static.oliveyoung.co.kr/pc-static-root...,55ml,모든 피부,"정제수, 부틸렌글라이콜, 세틸에틸헥사노에이트, 글리세린, 펜타에리스리틸테트라에틸헥사...","['엄마가 쓸 크림 찾다가 구매항건데 보습력 괜찮네요! 만족합니다', '피부가 너무..."
